In [1]:
!pip install openai
!pip install pinecone-client
!pip install langchain
!pip install tiktoken
!pip install pypdf

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/29/10/055b649e914ad8c5d07113c22805014988825abbeff007b0e89255b481fa/pypdf-3.17.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 2

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [3]:
# Load $AXSM's financial report. This may take 1-2 minutes since the PDF is large
axsm_10Q = "https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=317845829&type=PDF&symbol=AXSM&cdn=758892bcc180a6f1fb1b85181dfa0d06&companyName=Axsome+Therapeutics+Inc.&formType=10-Q&formDescription=General+form+for+quarterly+reports+under+Section+13+or+15%28d%29&dateFiled=2023-11-06"

# Create your PDF loader
loader = PyPDFLoader(axsm_10Q)

# Load the PDF document
axsm_documents = loader.load()

# Chunk the financial report
docs = text_splitter.split_documents(axsm_documents)
axsm_texts = [d.page_content for d in docs]

In [4]:
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone, Weaviate

/Users/arka_bagchi/opt/anaconda3/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
# The environment should be the one specified next to the API key
# in your Pinecone console
# Go to this link to create Pinecone index: https://app.pinecone.io/organizations/-Nlee1f4ZjpOS7ERdM6k/projects/gcp-starter:itrgkgp/indexes
pinecone.init(api_key="YOUR_PINECONE_API_KEY", environment="YOUR_PINECONE_ENVIRONMENT")
index = pinecone.Index("YOUR_PINECONE_INDEX")
openai_api_key = 'YOUR_OPENAI_API_KEY'
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore = Pinecone(index, embeddings, "text")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    ConfigurableField,
    RunnableBinding,
    RunnableLambda,
    RunnablePassthrough,
)

In [ ]:
# This is basic question-answering chain set up.
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(openai_api_key=openai_api_key)

retriever = vectorstore.as_retriever()

In [ ]:
# Here we mark the retriever as having a configurable field. All vectorstore retrievers have search_kwargs as a field. This is just a dictionary, with vectorstore specific fields
configurable_retriever = retriever.configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

In [ ]:
# Create the chain
chain = (
    {"context": configurable_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

Now, ask questions by ticker.

In [ ]:
chain.invoke(
    "What was revenue in July 2023?",
    config={"configurable": {"search_kwargs": {"namespace": "AXSM"}}},
)